In [127]:
from collections import defaultdict
from collections import OrderedDict

import pysradb
from pysradb import SRAdb
import os
import glob
import pandas as pd
from riboraptor.helpers import path_leaf, parse_star_logs, millify, order_dataframe
from riboraptor.cutadapt_to_json import cutadapt_to_json
from riboraptor.utils import summary_starlogs_over_runs, mkdir_p

root_dir = '/data1/re-ribo-analysis/'

builds = os.listdir(root_dir)

In [128]:
builds

['Mmul8',
 'GRCg6',
 'panTro3',
 'hg38',
 'Rnor6.0',
 'BDGP6',
 'GRCz11',
 'mm10',
 'WBcel235']

In [173]:
ROOT_DIRS = ["/data1/re-ribo-analysis", "/data4/re-ribo-analysis"]#, "/data3/re-ribo-analysis", "/data4/re-ribo-analysis"]
ROOT_DIRS_SUMMARY = ['/data2/re-ribo-analysis-summary-tables']

def check_ribotricer_output_exists(srp, srx, assembly):
    for rootdir in ROOT_DIRS:
        path = os.path.join(rootdir, assembly, srp, "ribotricer_results" ,"{}_translating_ORFs.tsv".format(srx))
        if os.path.exists(path):
            return path
    
def summarise_ribotricer_output_exists(path):
    df = pd.read_csv(path, sep='\t', use_cols = ['ORF_ID'])
    df_grouped 
    return df
    
def check_ribotricer_metagene_exists(srp, srx, assembly):
    for rootdir in ROOT_DIRS:
        path_5p = os.path.join(rootdir, assembly, srp, "ribotricer_results" ,"{}_metagene_profiles_5p.tsv".format(srx))
        path_3p = os.path.join(rootdir, assembly, srp, "ribotricer_results" ,"{}_metagene_profiles_3p.tsv".format(srx))
        path_5p_tsv = None
        path_3p_tsv = None
        if os.path.exists(path_5p):
            path_5p_tsv = path_5p
        if os.path.exists(path_3p):
            path_3p_tsv = path_3p
        if os.path.exists(path_5p) or os.path.exists(path_3p):
            return path_5p_tsv, path_3p_tsv
    return None, None
    



def check_ribotricer_metagene_plot_exists(srp, srx, assembly):
    for rootdir in ROOT_DIRS:
        path = os.path.join(rootdir, assembly, srp, "ribotricer_results" ,"{}_metagene_plots.pdf".format(srx))
        if os.path.exists(path):
            return path
    
def check_ribotricer_protocol_exists(srp, srx, assembly):
    for rootdir in ROOT_DIRS:
        path = os.path.join(rootdir, assembly, srp, "ribotricer_results" ,"{}_protocol.txt".format(srx))
        if os.path.exists(path):
            return path

def check_ribotricer_bam_summary_exists(srp, srx, assembly):
    for rootdir in ROOT_DIRS:
        path = os.path.join(rootdir, assembly, srp, "ribotricer_results" ,"{}_bam_summary.txt".format(srx))
        if os.path.exists(path):
            return path

def check_summarized_orfs_exists(srp, assembly):
    for rootdir in ROOT_DIRS_SUMMARY:
        path = os.path.join(rootdir, assembly, "{}_summarized_orfs.tsv".format(srp))
        if os.path.exists(path):
            return path
        

def check_summarized_phase_scores_exists(srp, assembly):
    for rootdir in ROOT_DIRS_SUMMARY:
        path = os.path.join(rootdir, assembly, "{}_summarized_phase_scores.tsv".format(srp))
        if os.path.exists(path):
            return path
        


In [151]:
def create_df_from_dir(rootdir):
    """Create a dataframe struture amenable fro ribotricer for samples with no metadata using their directory
    
    Parameters
    ----------
    path: string
          Directory location
    """
    srp = path_leaf(rootdir)
    samples = glob.glob('{}/ribotricer_results/*_translating_ORFs.tsv'.format(rootdir))
    samples = list(sorted([path_leaf(sample).replace('_translating_ORFs.tsv', '') for sample in samples]))
    df = []
    for sample in samples:
        df.append((srp, sample, sample))
    df = pd.DataFrame(df)
    df.columns = ['study_accession', 'experiment_accession', 'run_accession']
    df['library_layout'] = 'SINGLE'
    df["bases"] = ''
    df["spots"] = ''
    df['avg_read_length'] = ''
    df['library_source'] = ''
    df['library_selection'] = ''
    df['adapter_spec'] = ''
    df['library_strategy'] = ''
    df['library_name'] = ''
    df['experiment_title'] = ''
    df['taxon_id'] = ''
    return df


    
        

In [160]:
def get_srp_table(srp, assembly, re_ribo_analysis_dir):
    sradb = SRAdb("/data2/SRAmetadb.sqlite")
    column_order = [
        "study_accession",
        "experiment_title",
        "experiment_accession",
        "run_accession",
        "taxon_id",
        "library_selection",
        "library_layout",
        "library_strategy",
        "library_source",
        "library_name",
        "adapter_spec",
        "bases",
        "spots",
        "avg_read_length",
        "pass1_adapter",
        "pass1_total_reads_processed",
        "pass1_reads_with_adapters",
        "pass2_adapter",
        "pass2_total_reads_processed",
        "pass2_reads_with_adapters",
        "mapping_total_reads_input",
        "uniquely_mapped",
        "uniquely_mapped_percent",
        "ribotricer_orfs"
    ]
    filepath = os.path.join(re_ribo_analysis_dir, assembly, srp)
    if os.path.exists(filepath):

        try:
            srp_df = sradb.sra_metadata(srp.split("_")[0], detailed=True)#, expand_sample_attributes=True)
        except:
            srp_df = create_df_from_dir(filepath)
            
            #return pd.DataFrame()
        srp_df.library_layout = srp_df.library_layout.fillna("SINGLE")
        srp_df = srp_df[srp_df.library_layout.str.contains("SINGLE")]

        srp_df["pass1_reads_with_adapters"] = None
        srp_df["pass1_total_reads_processed"] = None
        srp_df["pass1_adapter"] = None
        srp_df["pass2_adapter"] = None
        srp_df["pass2_total_reads_processed"] = None
        srp_df["pass2_reads_with_adapters"] = None
        srp_df["mapping_total_reads_input"] = None
        srp_df["uniquely_mapped"] = None
        srp_df["uniquely_mapped_percent"] = None
        srp_df["ribotricer_orfs"] = None
        srp_df["ribotricer_metagene_5p"] = None
        srp_df["ribotricer_metagene_3p"] = None
        
        srp_df["ribotricer_metagene_plot"] =  None
        srp_df["ribotricer_protocol"] = None
        srp_df["ribotricer_bam_summary"] = None
        #srp_df["summarized_orfs"] = None
        #srp_df["summarized_phase_scores"] = None
        
        
        

        srpdir = os.path.join(re_ribo_analysis_dir, assembly, srp)
        starlogsdir = os.path.join(srpdir, "starlogs")
        srp_srx_grouped = srp_df.groupby("experiment_accession")
        preprocess_step1_dir = os.path.join(srpdir, "preprocessed_step1")
        preprocess_step2_dir = os.path.join(srpdir, "preprocessed")
        for srx, srx_group in srp_srx_grouped:
            ribotricer_output = check_ribotricer_output_exists(srp, srx, assembly)
            ribotricer_metagene_5p, ribotricer_metagene_3p = check_ribotricer_metagene_exists(srp, srx, assembly)
            
            ribotricer_bam_summary = check_ribotricer_bam_summary_exists(srp, srx, assembly)
            ribotricer_protocol = check_ribotricer_protocol_exists(srp, srx, assembly)            
            ribotricer_metagene_plot = check_ribotricer_metagene_plot_exists(srp, srx, assembly)
            
            #summarized_orfs = check_summarized_orfs_exists(srp, assembly)
            #summarized_phase_score = check_summarized_orfs_exists(srp, assembly)
            
            srrs = srx_group["run_accession"].tolist()
            if ribotricer_output:
                srp_df.loc[srp_df.experiment_accession == srx, "ribotricer_orfs"] = ribotricer_output
                
            srp_df.loc[srp_df.experiment_accession == srx, "ribotricer_metagene_5p"] = ribotricer_metagene_5p
            srp_df.loc[srp_df.experiment_accession == srx, "ribotricer_metagene_3p"] = ribotricer_metagene_3p

            srp_df.loc[srp_df.experiment_accession == srx, "ribotricer_bam_summary"] = ribotricer_bam_summary
            srp_df.loc[srp_df.experiment_accession == srx, "ribotricer_protocol"] = ribotricer_protocol
            srp_df.loc[srp_df.experiment_accession == srx, "ribotricer_metagene_plot"] = ribotricer_metagene_plot
            #srp_df.loc[srp_df.experiment_accession == srx, "summarized_orfs"] = summarized_orfs
            #srp_df.loc[srp_df.experiment_accession == srx, "summarized_phase_scores"] = summarized_phase_score

                
            # starlogs_df = summary_starlogs_over_runs(starlogsdir, srrs)

            for srr in srrs:
                starlogs_df = None
                if os.path.isfile(os.path.join(starlogsdir, srr + "Log.final.out")):
                    starlogs_df = parse_star_logs(
                        os.path.join(starlogsdir, srr + "Log.final.out")
                    )
                # Preprocessed_step1 adapter info
                step1_txt = os.path.join(
                    preprocess_step1_dir, srr + ".fastq.gz_trimming_report.txt"
                )
                step2_txt = os.path.join(
                    preprocess_step2_dir, srr + "_trimmed.fq.gz_trimming_report.txt"
                )
                step1_cutadapt_json = None
                step2_cutadapt_json = None

                if os.path.isfile(step1_txt):
                    step1_cutadapt_json = cutadapt_to_json(step1_txt)

                if os.path.isfile(step2_txt):
                    step2_cutadapt_json = cutadapt_to_json(step2_txt)

                if step1_cutadapt_json:
                    adapters = step1_cutadapt_json["adapters"]
                    if len(step1_cutadapt_json["adapters"]) == 0:
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass1_adapter"
                        ] = "Empty?"
                    elif isinstance(adapters, str):
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass1_adapter"
                        ] = step1_cutadapt_json["adapters"]
                    else:
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass1_adapter"
                        ] = step1_cutadapt_json["adapters"][
                            "{} - {}".format(srr, "Adapter 1")
                        ]
                        trim_info1 = step1_cutadapt_json["trim_info"][srr]
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass1_total_reads_processed"
                        ] = trim_info1["r_processed"]
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass1_reads_with_adapters"
                        ] = trim_info1["r_with_adapters"]
                if step2_cutadapt_json:
                    adapters = step2_cutadapt_json["adapters"]
                    if len(step2_cutadapt_json["adapters"]) == 0:
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass2_adapter"
                        ] = "Empty?"
                    elif isinstance(adapters, str):
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass2_adapter"
                        ] = step2_cutadapt_json["adapters"]
                    else:
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass2_adapter"
                        ] = step2_cutadapt_json["adapters"][
                            "{} - {}".format(srr + "_trimmed", "Adapter 1")
                        ]
                        trim_info2 = step2_cutadapt_json["trim_info"][srr + "_trimmed"]
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass2_reads_with_adapters"
                        ] = trim_info2["r_with_adapters"]
                        srp_df.loc[
                            srp_df.run_accession == srr, "pass2_total_reads_processed"
                        ] = trim_info2["r_processed"]

                if starlogs_df:
                    srp_df.loc[
                        srp_df.run_accession == srr, "mapping_total_reads_input"
                    ] = starlogs_df["total_reads"]
                    srp_df.loc[
                        srp_df.run_accession == srr, "uniquely_mapped"
                    ] = starlogs_df["uniquely_mapped"]
                    srp_df.loc[
                        srp_df.run_accession == srr, "uniquely_mapped_percent"
                    ] = starlogs_df["uniquely_mapped_percent"]

        cols = [
            "bases",
            "spots",
            "pass1_reads_with_adapters",
            "pass2_reads_with_adapters",
            "pass2_total_reads_processed",
            "pass1_total_reads_processed",
            "uniquely_mapped",
            "mapping_total_reads_input",
        ]
        for col in cols:
            try:
                srp_df[col] = srp_df[col].apply(lambda z: millify(z))
            except:
                pass
        sradb.close()
        return order_dataframe(srp_df, column_order)

In [161]:
READ_LENGTH_DIRNAME = "read_lengths"
METAGENE_COVERAGE_DIRNAME = "metagene_coverages"
METAGENE_LENWISE_COVERAGE_DIRNAME = "metagene_coverage_lengthwise"

# Top level directory of the directories inside each of the ROOT_DIRS
__ASSEMBLIES__ = [os.listdir(dirname) for dirname in ROOT_DIRS]
__SPECIES__ = [
    {"label": "H.sapiens", "value": "hg38"},
    {"label": "M.musculus", "value": "mm10"},
    {"label": "C.albicans", "value": "SC5314"}
]
__ASSEMBLIES__ = list(
    sorted(set([item for sublist in __ASSEMBLIES__ for item in sublist]))
)
__ASSEMBLY_WISE_SRP__ = defaultdict(list)
__SRP_TO_ROOT_DIR_MAP__ = defaultdict(dict)

#DATASETS = {"hg38": pd.read_csv("/data1/hg_datasets.tsv", sep="\t"),
#            "mm10": pd.read_csv("/data1/mm_datasets.tsv", sep="\t")}

for root_dir in ROOT_DIRS:
    for assembly_build in os.listdir(root_dir):
        for srp_dir in filter(
            os.path.isdir, glob.glob(os.path.join(root_dir, assembly_build, "*"))
        ):
            srp = os.path.basename(srp_dir)
            __ASSEMBLY_WISE_SRP__[assembly_build].append(srp)
            __SRP_TO_ROOT_DIR_MAP__[srp][assembly_build] = os.path.join(
                root_dir, assembly_build, srp
            )

def generate_tablex(dataframe, max_rows=26):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns]) ] +
        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )


In [162]:
__ASSEMBLY_WISE_SRP__ = defaultdict(list)
__SRP_TO_ROOT_DIR_MAP__ = defaultdict(dict)
for root_dir in ROOT_DIRS:
    for assembly_build in os.listdir(root_dir):
        for srp_dir in filter(
            os.path.isdir, glob.glob(os.path.join(root_dir, assembly_build, "*"))
        ):
            srp = os.path.basename(srp_dir)
            __ASSEMBLY_WISE_SRP__[assembly_build].append(srp)
            __SRP_TO_ROOT_DIR_MAP__[srp][assembly_build] = os.path.join(
                root_dir, assembly_build, srp
            )

In [163]:
__SRP_TO_ROOT_DIR_MAP__

defaultdict(dict,
            {'SRP028612': {'Mmul8': '/data1/re-ribo-analysis/Mmul8/SRP028612',
              'panTro3': '/data1/re-ribo-analysis/panTro3/SRP028612',
              'hg38': '/data1/re-ribo-analysis/hg38/SRP028612'},
             'SRP062129': {'Mmul8': '/data1/re-ribo-analysis/Mmul8/SRP062129',
              'panTro3': '/data1/re-ribo-analysis/panTro3/SRP062129',
              'hg38': '/data1/re-ribo-analysis/hg38/SRP062129'},
             'SRP096694': {'GRCg6': '/data1/re-ribo-analysis/GRCg6/SRP096694'},
             'SRP065528': {'hg38': '/data1/re-ribo-analysis/hg38/SRP065528'},
             'ERP021735': {'hg38': '/data1/re-ribo-analysis/hg38/ERP021735'},
             'SRP102021': {'hg38': '/data1/re-ribo-analysis/hg38/SRP102021'},
             'SRP065529': {'hg38': '/data1/re-ribo-analysis/hg38/SRP065529'},
             'SRP115659': {'hg38': '/data1/re-ribo-analysis/hg38/SRP115659'},
             'SRP044932': {'hg38': '/data1/re-ribo-analysis/hg38/SRP044932'},
      

In [164]:
__ASSEMBLY_WISE_SRP__

defaultdict(list,
            {'Mmul8': ['SRP028612', 'SRP062129'],
             'GRCg6': ['SRP096694'],
             'panTro3': ['SRP028612', 'SRP062129'],
             'hg38': ['SRP065528',
              'ERP021735',
              'SRP102021',
              'SRP065529',
              'SRP115659',
              'SRP044932',
              'SRP102616',
              'SRP103009',
              'SRP090415',
              'SRP044933',
              'SRP044935',
              'SRP075585',
              'SRP044936',
              'SRP058501',
              'SRP028612',
              'SRP102020',
              'SRP062129',
              'SRP113333',
              'SRP065530',
              'SRP083699',
              'SRP114321',
              'SRP044934',
              'SRP067300',
              'SRP044937',
              'SRP059546',
              'SRP101952',
              'SRP098789',
              'SRP059547',
              'SRP062129_rm_quicksect',
              'SRP059548',
            

In [165]:
def get_fragment_lengths(file_path):
    try:
        return pd.read_csv(file_path, sep='\t').fragment_length.tolist()
    except:
        # Handle 3 headed files
        df = pd.read_csv(file_path, header=None, sep='\t')
        df.columns = ['fragment_length', 'offset_5p', 'profile']
        return df.fragment_length.tolist()


In [214]:
db = SRAdb('/data2/SRAmetadb.sqlite')
all_projects = []
re_ribo_analysis_dir = '/data1/re-ribo-analysis'


for species, sample_list in __ASSEMBLY_WISE_SRP__.items():
    mkdir_p('/data2/re-ribo-analysis-metadata/{}'.format(species))
    for srp in sample_list:
        basedir = os.path.dirname(os.path.dirname(__SRP_TO_ROOT_DIR_MAP__[srp][species]))
        df = get_srp_table(srp, species, basedir)
        project_filepath = '{}/{}/{}'.format(basedir, species, srp)    
        metadata_filepath = '/data2/re-ribo-analysis-metadata/{}/{}.tsv'.format(species, srp)
        df_subset = df[df.ribotricer_metagene_5p == df.ribotricer_metagene_5p].ribotricer_metagene_5p.tolist()
        summarized_orfs = check_summarized_orfs_exists(srp, species)
        summarized_phase_score = check_summarized_phase_scores_exists(srp, species)
        fragment_lengths = []
        for f in df_subset:
            fragment_lengths += get_fragment_lengths(f)
        fragment_lengths = list(sorted(list(set(fragment_lengths))))
        all_projects.append((species, srp, project_filepath, metadata_filepath, str(fragment_lengths), summarized_orfs, summarized_phase_score))        
        df.to_csv(metadata_filepath, sep='\t', index=False, header=True)

In [215]:
df = pd.read_csv("/data2/re-ribo-analysis-summary-tables/hg38/SRP113333_summarized_orfs.tsv", sep="\t")
df = pd.pivot_table(df, columns=['ORF_type', 'status'], index='experiment_accession', values=['count'])['count']
df

ORF_type                  annotated                       dORF              \
status               nontranslating translating nontranslating translating   
experiment_accession                                                         
SRX3028093                    75048       11958          23983        1237   
SRX3028094                    68868       18138          23754        1466   
SRX3028095                    64212       22794          22896        2324   
SRX3028096                    68899       18107          23652        1568   
SRX3028097                    83950        3056          23653        1567   
SRX3028098                    82464        4542          24399         821   
SRX3028099                    84370        2636          23491        1729   
SRX3028100                    82696        4310          24317         903   
SRX3028101                    83568        3438          23773        1447   
SRX3028102                    83394        3612          24008        1212   
SRX3028103                    81925        5081          24950         270   
SRX3028104                    84286        2720          23638        1582   
SRX3028105                    47576       39430          23111        2109   
SRX3028106                    60259       26747          23293        1927   
SRX3028107                    53235       33771          23488        1732   
SRX3028108                    57338       29668          22981        2239   

ORF_type                      novel               overlap_dORF              \
status               nontranslating translating nontranslating translating   
experiment_accession                                                         
SRX3028093                   123809        4984           8019         874   
SRX3028094                   123409        5384           7764        1129   
SRX3028095                   120306        8487           7236        1657   
SRX3028096                   122704        6089           7681        1212   
SRX3028097                   125158        3635           8293         600   
SRX3028098                   126463        2330           8418         475   
SRX3028099                   124744        4049           8282         611   
SRX3028100                   126514        2279           8429         464   
SRX3028101                   125424        3369           8290         603   
SRX3028102                   125770        3023           8390         503   
SRX3028103                   127729        1064           8606         287   
SRX3028104                   125000        3793           8301         592   
SRX3028105                   120823        7970           7171        1722   
SRX3028106                   121778        7015           7404        1489   
SRX3028107                   122090        6703           7528        1365   
SRX3028108                   120673        8120           7277        1616   

ORF_type               overlap_uORF                 super_dORF              \
status               nontranslating translating nontranslating translating   
experiment_accession                                                         
SRX3028093                     3658         556          78130         104   
SRX3028094                     3660         554          78154          80   
SRX3028095                     3447         767          78064         170   
SRX3028096                     3652         562          78163          71   
SRX3028097                     3914         300          73572        4662   
SRX3028098                     3926         288          76349        1885   
SRX3028099                     3902         312          72946        5288   
SRX3028100                     3965         249          75968        2266   
SRX3028101                     3881         333          74643        3591   
SRX3028102                     3876         338          75443        2791   
SRX3028103          

In [216]:
summary_df = pd.DataFrame(all_projects)
summary_df.columns = ['species', 'srp', 'project_output_path', 'project_metadata_path', 'fragment_lengths', 'summarized_orfs', 'summarized_phase_scores']
summary_df = summary_df.sort_values(by=['species', 'srp'])
summary_df.to_csv('/data2/datasets.tsv', sep='\t', index=False, header=True)
summary_df

,species,srp,project_output_path,project_metadata_path,fragment_lengths,summarized_orfs,summarized_phase_scores
40,BDGP6,ERP008887,/data1/re-ribo-analysis/BDGP6/ERP008887,/data2/re-ribo-analysis-metadata/BDGP6/ERP0088...,[],None,None
42,BDGP6,SRP028243,/data1/re-ribo-analysis/BDGP6/SRP028243,/data2/re-ribo-analysis-metadata/BDGP6/SRP0282...,"[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 3...",/data2/re-ribo-analysis-summary-tables/BDGP6/S...,/data2/re-ribo-analysis-summary-tables/BDGP6/S...
43,BDGP6,SRP033366,/data1/re-ribo-analysis/BDGP6/SRP033366,/data2/re-ribo-analysis-metadata/BDGP6/SRP0333...,"[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...",/data2/re-ribo-analysis-summary-tables/BDGP6/S...,/data2/re-ribo-analysis-summary-tables/BDGP6/S...
44,BDGP6,SRP072369,/data1/re-ribo-analysis/BDGP6/SRP072369,/data2/re-ribo-analysis-metadata/BDGP6/SRP0723...,"[34, 35, 36, 37, 38]",/data2/re-ribo-analysis-summary-tables/BDGP6/S...,/data2/re-ribo-analysis-summary-tables/BDGP6/S...
41,BDGP6,SRP108999,/data1/re-ribo-analysis/BDGP6/SRP108999,/data2/re-ribo-analysis-metadata/BDGP6/SRP1089...,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",/data2/re-ribo-analysis-summary-tables/BDGP6/S...,/data2/re-ribo-analysis-summary-tables/BDGP6/S...
2,GRCg6,SRP096694,/data1/re-ribo-analysis/GRCg6/SRP096694,/data2/re-ribo-analysis-metadata/GRCg6/SRP0966...,"[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2...",/data2/re-ribo-analysis-summary-tables/GRCg6/S...,/data2/re-ribo-analysis-summary-tables/GRCg6/S...
49,GRCz11,SRP010040,/data1/re-ribo-analysis/GRCz11/SRP010040,/data2/re-ribo-analysis-metadata/GRCz11/SRP010...,"[19, 20, 21, 23, 24]",None,None
46,GRCz11,SRP021915,/data1/re-ribo-analysis/GRCz11/SRP021915,/data2/re-ribo-analysis-metadata/GRCz11/SRP021...,"[23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",/data2/re-ribo-analysis-summary-tables/GRCz11/...,/data2/re-ribo-analysis-summary-tables/GRCz11/...
47,GRCz11,SRP023492,/data1/re-ribo-analysis/GRCz11/SRP023492,/data2/re-ribo-analysis-metadata/GRCz11/SRP023...,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",/data2/re-ribo-analysis-summary-tables/GRCz11/...,/data2/re-ribo-analysis-summary-tables/GRCz11/...
48,GRCz11,SRP033369,/data1/re-ribo-analysis/GRCz11/SRP033369,/data2/re-ribo-analysis-metadata/GRCz11/SRP033...,"[19, 20, 24, 25, 29, 33, 34, 35, 36, 39]",/data2/re-ribo-analysis-summary-tables/GRCz11/...,/data2/re-ribo-analysis-summary-tables/GRCz11/...


In [169]:
summary_df[summary_df.summarized_orfs == summary_df.summarized_orfs]

,species,srp,project_output_path,project_metadata_path,fragment_lengths,summarized_orfs,summarized_phase_scores


In [84]:
summary_df.loc[summary_df.srp=='Kadosh_30C_37C'].iloc[0]

species                                                             SC5314
srp                                                         Kadosh_30C_37C
project_output_path          /data4/re-ribo-analysis/SC5314/Kadosh_30C_37C
project_metadata_path    /data2/re-ribo-analysis-metadata/SC5314/Kadosh...
fragment_lengths         [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3...
Name: 81, dtype: object

In [86]:
pd.read_csv('/data2/re-ribo-analysis-metadata/SC5314/Kadosh_30C_37C.tsv', sep='\t')

,study_accession,experiment_title,experiment_accession,run_accession,taxon_id,library_selection,library_layout,library_strategy,library_source,library_name,...,pass2_reads_with_adapters,mapping_total_reads_input,uniquely_mapped,uniquely_mapped_percent,ribotricer_orfs,ribotricer_metagene_5p,ribotricer_metagene_3p,ribotricer_metagene_plot,ribotricer_protocol,ribotricer_bam_summary
0,Kadosh_30C_37C,NaN,ribo_30C_1,ribo_30C_1,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
1,Kadosh_30C_37C,NaN,ribo_30C_2,ribo_30C_2,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
2,Kadosh_30C_37C,NaN,ribo_30C_3,ribo_30C_3,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
3,Kadosh_30C_37C,NaN,ribo_37C_1,ribo_37C_1,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
4,Kadosh_30C_37C,NaN,ribo_37C_2,ribo_37C_2,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
5,Kadosh_30C_37C,NaN,ribo_37C_3,ribo_37C_3,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
6,Kadosh_30C_37C,NaN,rna_30C_1,rna_30C_1,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
7,Kadosh_30C_37C,NaN,rna_30C_2,rna_30C_2,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
8,Kadosh_30C_37C,NaN,rna_30C_3,rna_30C_3,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...,/data4/re-ribo-analysis/SC5314/Kadosh_30C_37C/...
9,Kadosh_30C_37C,NaN,rna_37C_1,rna_37C_1,NaN,NaN,SINGLE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,/data4/re-ribo-analysis/SC

In [137]:
%%timeit 
ribotricer_df = pd.read_csv("/data1/re-ribo-analysis/hg38/SRP044936/ribotricer_results/SRX663288_translating_ORFs.tsv", sep='\t')


/home/saket/anaconda3/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
/home/saket/anaconda3/lib/python3.6/site-packages/IPython/core/magics/execution.py:1102: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  all_runs = timer.repeat(repeat, number)


27.1 s ± 157 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [140]:
ribotricer_df = pd.read_csv("/data1/re-ribo-analysis/hg38/SRP044936/ribotricer_results/SRX663288_translating_ORFs.tsv", sep='\t')

ribotricer_df.head()

/home/saket/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ORF_ID,ORF_type,status,phase_score,read_count,length,valid_codons,transcript_id,transcript_type,gene_id,gene_name,gene_type,chrom,strand,start_codon,profile
0,ENST00000641515_65565_70005_978,annotated,nontranslating,0.000000,0,978,0,ENST00000641515,protein_coding,ENSG00000186092,OR4F5,protein_coding,1,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,ENST00000335137_69091_70005_915,annotated,nontranslating,0.000000,0,915,0,ENST00000335137,protein_coding,ENSG00000186092,OR4F5,protein_coding,1,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,ENST00000426406_450743_451678_936,annotated,nontranslating,0.000000,0,936,0,ENST00000426406,protein_coding,ENSG00000284733,OR4F29,protein_coding,1,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,ENST00000332831_685719_686654_936,annotated,nontranslating,0.000000,0,936,0,ENST00000332831,protein_coding,ENSG00000284662,OR4F16,protein_coding,1,-,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,ENST00000420190_924432_939291_1074,annotated,nontranslating,0.330719,9,1074,8,ENST00000420190,protein_coding,ENSG00000187634,SAMD11,protein_coding,1,+,ATG,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [143]:
%timeit ribotricer_df = pd.read_csv("/data1/re-ribo-analysis/hg38/SRP044936/ribotricer_results/SRX663288_translating_ORFs.tsv", sep='\t', usecols=['ORF_type', 'status', 'phase_score'])


15 s ± 23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
ORF_TYPES = ribotricer_df.ORF_type.unique()

In [99]:
ribotricer_df = pd.read_csv("/data1/re-ribo-analysis/hg38/SRP044936/ribotricer_results/SRX663288_translating_ORFs.tsv", sep='\t', usecols=[ 'ORF_type', 'status', 'phase_score', 'start_codon'])
ribotricer_df = ribotricer_df.loc[ribotricer_df.start_codon=='ATG']
ribotricer_df_grouped = ribotricer_df.groupby(['ORF_type', 'status'])

import tqdm

In [124]:
def summarize_ribotrocer_orf_project(rootdir):    
    srp = path_leaf(rootdir)
    srp = path_leaf(rootdir)
    samples = glob.glob('{}/ribotricer_results/*_translating_ORFs.tsv'.format(rootdir))
    summarized_orf_data = []
    summarized_phase_scores_df = pd.DataFrame()
    for sample_index, sample in enumerate(samples):
        srx = path_leaf(sample).replace('_translating_ORFs.tsv', '')
        ribotricer_df = pd.read_csv(sample, sep='\t', usecols=[ 'ORF_ID', 'ORF_type', 'status', 'phase_score', 'start_codon', 'phase_score'])
        ribotricer_df = ribotricer_df.loc[ribotricer_df.start_codon=='ATG']
        
        if sample_index == 0:
            summarized_phase_scores_df = ribotricer_df[['ORF_ID', 'phase_score']]#.set_index('ORF_ID')
            summarized_phase_scores_df.columns = ['ORF_ID', srx]
        else:
            phase_score_df = ribotricer_df[['phase_score']].rename(columns={'phase_score': srx})
            summarized_phase_scores_df = pd.concat([summarized_phase_scores_df, phase_score_df], axis=1)            

    ribotricer_df_grouped = ribotricer_df.groupby(['ORF_type', 'status'])
    for group, df in ribotricer_df_grouped:
        summarized_orf_data.append((srx, group[0], group[1], df.shape[0]))
    
    summarized_orf_data = pd.DataFrame(summarized_orf_data, columns=['experiment_accession', 'ORF_type', 'status', 'count'])
    return summarized_orf_data, summarized_phase_scores_df


In [148]:
rootdir = '/data1/re-ribo-analysis/hg38/SRP098789/'
%time summarized_orf_data, summarized_phase_scores_df = summarize_ribotrocer_orf_project(rootdir)

CPU times: user 5min 58s, sys: 16.4 s, total: 6min 15s
Wall time: 7min 17s


In [149]:
summarized_orf_data

[('SRX2536408', 'annotated', 'nontranslating', 36720),
 ('SRX2536408', 'annotated', 'translating', 50286),
 ('SRX2536408', 'dORF', 'nontranslating', 21851),
 ('SRX2536408', 'dORF', 'translating', 3369),
 ('SRX2536408', 'novel', 'nontranslating', 116799),
 ('SRX2536408', 'novel', 'translating', 11994),
 ('SRX2536408', 'overlap_dORF', 'nontranslating', 6223),
 ('SRX2536408', 'overlap_dORF', 'translating', 2670),
 ('SRX2536408', 'overlap_uORF', 'nontranslating', 3240),
 ('SRX2536408', 'overlap_uORF', 'translating', 974),
 ('SRX2536408', 'super_dORF', 'nontranslating', 78106),
 ('SRX2536408', 'super_dORF', 'translating', 128),
 ('SRX2536408', 'super_uORF', 'nontranslating', 7028),
 ('SRX2536408', 'super_uORF', 'translating', 231),
 ('SRX2536408', 'uORF', 'nontranslating', 3879),
 ('SRX2536408', 'uORF', 'translating', 419)]

In [ ]:
summarized_orf_data.to_csv('/data2/re-ribo-analysis-orf-tables/')

In [121]:
rootdir = '/data1/re-ribo-analysis/hg38/SRP098789/'
srp = path_leaf(rootdir)
samples = glob.glob('{}/ribotricer_results/*_translating_ORFs.tsv'.format(rootdir))
# = list(sorted([path_leaf(sample).replace('_translating_ORFs.tsv', '') for sample in samples]))
summarized_orf_data = []
summarized_phase_scores_df = pd.DataFrame()
for sample_index, sample in enumerate(samples):
    srx = path_leaf(sample).replace('_translating_ORFs.tsv', '')
    ribotricer_df = pd.read_csv(sample, sep='\t', usecols=[ 'ORF_ID', 'ORF_type', 'status', 'phase_score', 'start_codon', 'phase_score'])
    ribotricer_df = ribotricer_df.loc[ribotricer_df.start_codon=='ATG']

    if sample_index == 0:
        summarized_phase_scores_df = ribotricer_df[['ORF_ID', 'phase_score']]#.set_index('ORF_ID')
        summarized_phase_scores_df.columns = ['ORF_ID', srx]
    else:
        phase_score_df = ribotricer_df[['phase_score']].rename(columns={'phase_score': srx})
        summarized_phase_scores_df = pd.concat([summarized_phase_scores_df, phase_score_df], axis=1)            

    ribotricer_df_grouped = ribotricer_df.groupby(['ORF_type', 'status'])
    for group, df in ribotricer_df_grouped:
        summarized_orf_data.append((srx, group[0], group[1], df.shape[0]))

In [146]:
summarized_phase_scores_df.head()

,ORF_ID,SRX2536424,SRX2536427,SRX2536409,SRX2536419,SRX2536428,SRX2536426,SRX2536410,SRX2536414,SRX2536415,...,SRX2536417,SRX2536406,SRX2536421,SRX2536403,SRX2536416,SRX2536422,SRX2536407,SRX2536425,SRX2536418,SRX2536408
0,ENST00000641515_65565_70005_978,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,ENST00000335137_69091_70005_915,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,ENST00000426406_450743_451678_936,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,ENST00000332831_685719_686654_936,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,ENST00000420190_924432_939291_1074,0.444444,0.057735,0.377964,0.5,0.5,0.277732,1.0,1.0,0.433013,...,0.515079,1.0,0.57735,1.0,1.0,1.0,1.0,0.166667,0.5,0.5


In [123]:
summarized_phase_scores_df.head()

,ORF_ID,SRX2536424,SRX2536427,SRX2536409,SRX2536419,SRX2536428,SRX2536426,SRX2536410,SRX2536414,SRX2536415,...,SRX2536417,SRX2536406,SRX2536421,SRX2536403,SRX2536416,SRX2536422,SRX2536407,SRX2536425,SRX2536418,SRX2536408
0,ENST00000641515_65565_70005_978,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,ENST00000335137_69091_70005_915,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,ENST00000426406_450743_451678_936,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,ENST00000332831_685719_686654_936,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,ENST00000420190_924432_939291_1074,0.444444,0.057735,0.377964,0.5,0.5,0.277732,1.0,1.0,0.433013,...,0.515079,1.0,0.57735,1.0,1.0,1.0,1.0,0.166667,0.5,0.5


In [94]:
def read_ribotricer_bam_summary(file_path):
    summary_dict = OrderedDict()
    fragment_len_dist_dict = OrderedDict()
    reading_length_dist = False
    reading_summary = False
    with open(file_path) as fh:
        for index, line in enumerate(fh):
            line = line.strip()
            if line == '':
                continue
            if index == 0:
                assert line == 'summary:'
                reading_summary = True
                continue
            if line == 'length dist:':
                reading_summary = False
                reading_length_dist = True
                continue

            if reading_summary:
                try:
                    key, value = line.split(':')
                except:
                    raise Exception('Unable to parse {}'.format(line))
                value = value.strip(' ')
                summary_dict[key] = int(value)
            if reading_length_dist:
                try:
                    key, value = line.split(':')
                except:
                    raise Exception('Unable to parse {}'.format(line))
                value = value.strip(' ')
                fragment_len_dist_dict[int(key)] = int(value)
    return summary_dict, pd.Series(fragment_len_dist_dict).sort_index()
                
            
            

# Summarisze count files

In [ ]:
def summary_read_count_file(file_path):
    """Read a counts file outputted by ribotircer and get the sum of the counts
    
    Parameters
    ----------
    file_path: string
    
    Returns
    -------
    dist_normalized_counts: array
                            Array of counts normalized by length
    sum_counts: int
                Counts
    """
    df = pd.read_csv(file_path, sep='\t')
    normalized = df.count/df.length
    return df.count.sum(), normalized

def 


In [119]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b

In [122]:
rgb(0.42, 1, 0.42)
import seaborn as sns

In [131]:
sns.light_palette((260, 75, 60), input="husl")
def hex_to_rgb(h):
    h = h.lstrip('#')
    return 'rgb({}, {}, {})'.format(*tuple(int(h[i:i+2], 16) for i in (0, 2, 4)))

In [132]:
hex_to_rgb('#eaedfb')

'rgb(234, 237, 251)'

In [136]:
from matplotlib.colors import LinearSegmentedColormap

boundaries = [0.0, 0.42, 0.5, 0.6, 0.7, 0.7, 0.9, 1.0]  # custom boundaries

# here I generated twice as many colors, 
# so that I could prune the boundaries more clearly
hex_colors = sns.light_palette((260, 75, 60), input="husl", n_colors=len(boundaries) * 2 + 2, as_cmap=False).as_hex()
hex_colors = [hex_to_rgb(hex_colors[i]) for i in range(0, len(hex_colors), 2)]

#rgb_colors = sns.light_palette((260, 75, 60), input="husl", n_colors=len(boundaries) * 2 + 2, as_cmap=False).as_rgb()
#rgb_colors = [hex_to_rgb(rgb_colors[i]) for i in range(0, len(rgb_colors), 2)]

colors=list(zip(boundaries, hex_colors))
colors


[(0.0, 'rgb(234, 237, 251)'),
 (0.42, 'rgb(220, 226, 248)'),
 (0.5, 'rgb(207, 214, 245)'),
 (0.6, 'rgb(193, 203, 243)'),
 (0.7, 'rgb(180, 191, 240)'),
 (0.7, 'rgb(166, 180, 237)'),
 (0.9, 'rgb(153, 168, 234)'),
 (1.0, 'rgb(139, 157, 232)')]

In [125]:
custom_color_map

In [134]:
colors

[(0.0, '#eaedfb'),
 (0.42, '#dce2f8'),
 (0.5, '#cfd6f5'),
 (0.6, '#c1cbf3'),
 (0.7, '#b4bff0'),
 (0.7, '#a6b4ed'),
 (0.9, '#99a8ea'),
 (1.0, '#8b9de8')]